<a href="https://colab.research.google.com/github/sbertolin/03MAIR-Algoritmos-de-optimizacion/blob/master/SEMINARIO/Seminario_Santiago_Bertolin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:  Santiago Bertolín Martínez <br>
Url: https://github.com/sbertolin/03MAIR-Algoritmos-de-optimizacion/blob/master/SEMINARIO<br>
Problema:
> ### 2. Organizar los horarios de partidos de La Liga<br>

Descripción del problema:<br>

Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios que maximice la audiencia. <br>

Se conoce estadísticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos). <br>

En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores que tienen relación directa con la audiencia. Hay 4 equipos en la categoría A,10 equipos de categoría B y 6 equipos de categoría C.<br>

> A vs A: 2M<br>
> A vs B: 1,3M<br>
> A vs C: 1M<br>
> B vs B: 0,9M<br>
> B vs C: 0,75M<br>
> C vs C: 0,47M<br>

Los horarios disponibles se conocen a priori, son los siguientes y tienen la siguiente relación de audiencia con la del de sábado a las 20h:<br>

> Viernes: 20:00 (0,4)<br>
> Sábado: 12:00 (0,55); 16:00 (0,7); 18:00 (0,8); 20:00 (1)<br>
> Domingo: 12:00 (0,45); 16:00 (0,75); 18:00 (0,85); 20:00 (1)<br>
> Lunes: 20:00 (0,4)<br>

Es posible la coincidencia de horarios pero en este caso la audiencia se verá afectada(no se acumula) y se repartirá según los siguientes criterios:<br>
> Se tomará la mejor audiencia y se repartirá según la siguiente ponderación: A = 4 puntos, B = 2 puntos, C = 1 punto<br>
> Si hay una coincidencia de horario para dos partidos por ejemplo: AB y BC<br>
>> Total puntos: 4(A)+2(B)  +  2(B)+1(C) =9<br>
>> La audiencia máxima para ese horario será la de AB, 1.3
>> Para AB: 0,87M
>> Para BC: 0,43M

                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



(*)¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




**Respuesta**

*Antes de empezar a responder esta pregunta y el resto de apartados, comentar que, con las condiciones dadas, en ningún caso será óptima la coincidencia de partidos en el mismo horario. Esto se debe a que el número de espectadores de los dos partidos combinados será el mismo al de 1 solo partido (el de mayor audiencia) por lo que la aportación del otro será de 0. Siendo así, cualquier otro horario para este segundo partido será mejor. <br>
Teniendo esto en cuenta, a partir de este punto, se despreciará la posibilidad de coincidencia y se considerará que se asigna un partido a cada uno de los horarios posibles.*<br><br>

Si el problema se reduce a asignar los 10 partidos a los 10 horarios posibles sin repetición, lo podemos asemejar a buscar las posibles permutaciones de la lista de 10 partidos.

Siendo así, el número de posibles soluciones será de 10! = 3628800

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


**Respuesta**

En el caso partícular con los condicionantes ya descritos, se ha decidido modelizar el problema como uno de Programación Lineal Entera, con una función lineal de salida a maximizar a partir de unos coeficientes condicionados (se describirán en el siguiente apartado). Parte de estos coeficientes se representan como una matriz de audiencias. Por una lado porque nos permite un acceso con coste 1 a cada uno de los valores de audiencia posibles según partido y horario, y por otro porque nos permite visualizar el problema de forma muy esclarecedora.<br><br>

Exactamente, se podría decir que és un 
[*Problema de Asignación:*](https://es.wikipedia.org/wiki/Problema_de_la_asignaci%C3%B3n)<br><br>

Más adelante, en el último punto, en el caso en que se tuviera que llevar a cabo la resolución del problema como se describirá en ese apartado, se utilizarían árboles de expansión.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

**Respuesta**

Como ya se ha comentado, nos encontramos con un problema de **maximización** dónde la función objetivo a maximizar és la siguiente:

$$max(\sum_{j=0}^{N-1} \sum_{k=0}^{N-1} \alpha_{jk} h_{jk})$$

Dónde $h_{jk}$ són los coeficientes de la matriz de audiencias (ver más abajo) y $\alpha_{jk} = \{0, 1\}$ son las elecciones de partido y horario, que han de cumplir:
$$\sum_{j=0}^{N-1} \alpha_{jk} = 1 \hspace{0.3cm} \forall k$$
$$\sum_{k=0}^{N-1} \alpha_{jk} = 1 \hspace{0.3cm} \forall j$$

(*)Diseña un algoritmo para resolver el problema por fuerza bruta

**Respuesta**

Se diseña un algoritmo que recorre y compara todas las posibles soluciones sin ningún filtro ni condicionante.<br><br>
Antes de empezar, se ejecuta una celda para inicializar las constantes que definen el problema.

In [0]:
# Se importan las librerías que se utilizarán a lo largo de los ejercicios
import random
import numpy as np
import pandas as pd

# Se define una función para encontrar la audiencia base de un partido entre los
# equipos 1 y 2
AUDIENCES_TEAMS = [[2, 1.3, 1], [1.3, 0.9, 0.75], [1, 0.75, 0.47]]
LETTER_DICT = {'A': 0, 'B': 1, 'C': 2}
def match_audience(team_1, team_2):
  return AUDIENCES_TEAMS[LETTER_DICT[team_1]][LETTER_DICT[team_2]]

# Se define la lista de multiplicadores según horario
TIMETABLE_MULT = [
    [0.4,  'Viernes: 20:00', 'V20 (0.4)', 0],
    [0.55, 'Sábado:  12:00', 'S12 (0.55)', 1],
    [0.7,  'Sábado:  16:00', 'S16 (0.7)', 2],
    [0.8,  'Sábado:  18:00', 'S18 (0.8)', 3],
    [1,    'Sábado:  20:00', 'S20 (1)', 4],
    [0.45, 'Domingo: 12:00', 'D12 (0.45)', 5],
    [0.75, 'Domingo: 16:00', 'D16 (0.75)', 6],
    [0.85, 'Domingo: 18:00', 'D18 (0.85)', 7],
    [1,    'Domingo: 20:00', 'D20 (1)', 8],
    [0.4,  'Lunes:   20:00', 'L20 (0.4)', 9]
]

# Se define la lista de equipos en la liga actual según el volumen de sus aficiones
TEAMS = ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C']

In [0]:
# Se define una función para calcular la audiencia dada una solución posible.
# Esta es la función a maximizar por el algoritmo.
# La solución es una lista dónde la posición es el partido y el valor en esa posición
# el horario asociado al partido.
def calculate_audience(audience_matrix, test_solution):
  total_audience = 0
  for i in range(len(test_solution)):
    total_audience += audience_matrix[i][test_solution[i]]
    
  return total_audience

# Función recursiva que nos permite recorrer todas las permutaciones posibles de
# la lista de horarios sobre la lista de partidos.
# En res_list tenemos la solución a provar que se va generando iterativamente.
# max_audience y best_solution se van pasando por "referencia" de forma que se pueden
# ir utilizando para combrobar si se está en la mejor solución o no.
def calc_permutations(audience_matrix, pos_list, res_list, max_audience, best_solution):
  
  # Si se cumple la condición quiere decir que ya se ha formado una de las posibles soluciones
  # y se ha de comprobar si esta es la mejor.
  if len(pos_list) == 1:
    new_res_list = res_list + [pos_list[0]]
    res_audience = calculate_audience(audience_matrix, new_res_list)
    
    # Si la solución es la mejor hasta el momento, se guarda en las referencias de mejor
    # solución
    if res_audience > max_audience[0]:
      
      # max_audience es una lista ya que los enteros no son mutables en python y siempre
      # se pasan por copia
      max_audience[0] = res_audience
      best_solution[:] = new_res_list
      
  # En este caso aún no se ha llegado a formar una solución y se pasa una de las posiciones de
  # de la lista de entrada a la lista de la solución.
  else:
    for i in range(len(pos_list)): 
      new_rest_list = res_list + [pos_list[i]]
      new_pos_list = pos_list[:i] + pos_list[i+1:]
      calc_permutations(audience_matrix, new_pos_list, new_rest_list, max_audience, best_solution)

# Punto de entrada del algoritmo de solución por fuerza bruta
def brute_force(audience_matrix):
  best_solution = []
  max_audience = [0]
  initial_positions = [i for i in range(len(audience_matrix))]
  
  calc_permutations(audience_matrix, initial_positions, [], max_audience, best_solution)

  return best_solution, max_audience

(*)Calcula la complejidad del algoritmo por fuerza bruta

**Respuesta**

Empezamos calculando la complejidad de *calculate_audience*:<br>
> 3n + 3<br>

Después podemos ver que, para cada una de las soluciones, además, se realizan 4 operaciones y n+1 más si el resultado es el mejor hasta el momento.<br>
Durante las iteraciones se realizan más operaciones, pero son despreciables en comparación con las que multiplican el factorial. <br><br>
En resumen, el número de operaciones aproximadas serán de **n!(4n+k)** con un órden de complejidad de **O(n!)**.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

**Respuesta**

Como se ha comentado al principio de la práctica, las condiciones aplicadas al problema lo simplifican mucho llevando a poderse resolver con una voracidad extrema.<br><br>
Si se analiza el problema, debido a que las condiciones son simplemente unos coeficientes multiplicativos, se puede llegar a concluir que la mejor solución se consigue siempre multiplicando el partido de mayor audiencia por el horario con mejor coeficiente multiplicativo, el segundo mejor partido por el segundo mejor coeficiente y así en adelante. <br><br>
Esto se transformaría en:
$$\forall a > b ;  c > d\Rightarrow ac +bd > ad +bc$$<br>
Y se puede demostrar de la siguiente manera:
$$ac+bd-ad-bc > 0$$
$$a(c-d)+b(d-c) > 0$$
$$k = c-d > 0 \Rightarrow ak-bk >0 \Rightarrow a-b > 0$$<br>
Se realizaran dos algoritmos, uno utilizando directamente la ordenación de las dos listas, y otro, no tan óptimo a nivel computacional, pero que seguirá un esquema similar a los algoritmos voraces vistos hasta ahora.

In [0]:
# Algoritmo por cruce de listas ordenadas
def sort_algorithm(audience_list, audience_matrix):
  
  length = len(audience_list)
  best_solution = [0 for i in range(length)]

  # Se ordena la lista de partidos por audiencia descendente
  audience_sorted_by_value = sorted(audience_list.items(), key=lambda x: x[1], reverse=True)
  
  # Se ordena la lista (matriz) de horarios por multiplicador de audiencia descendente
  times_sorted_by_value = sorted(TIMETABLE_MULT, key=lambda x: x[0], reverse=True)
 
  # Como se ha explicado previamente se calcula la mejor solución multiplicando el mejor
  # partido por el mejor coeficiente, el segundo mejor partido por el segundo mejor
  # coeficiente y así en adelante
  for i in range(length):
    best_solution[audience_sorted_by_value[i][0]] = times_sorted_by_value[i][3]
  
  # Se calcula la audiencia para la solución encontrada
  max_audience = calculate_audience(audience_matrix, best_solution)

  return best_solution, max_audience

# Algoritmo similar al anterior, estilo algoritmo voraz
def greedy_algorithm(audience_list, audience_matrix):
  
  length = len(audience_list)
  best_solution = [0 for i in range(length)]
  
  # Se ordena la lista (matriz) de horarios por multiplicador de audiencia descendente
  times_sorted_by_value = sorted(TIMETABLE_MULT, key=lambda x: x[0], reverse=True)
  
  for i in range(length):
    max_val = 0
    it = 0
    for key in audience_list:
      if audience_list[key] > max_val:
        max_val = audience_list[key]
        it = key
        
    del audience_list[it]
    best_solution[it] = times_sorted_by_value[i][3]
  
  max_audience = calculate_audience(audience_matrix, best_solution)

  return best_solution, max_audience

(*)Calcula la complejidad del algoritmo 

**Respuesta**

El algoritmo *sort_algorithm* utiliza la función *calculate_audience* (aunque no tendría porqué formar parte del algoritmo), con un coste de 3n + 3, el bucle, que serán también 3n y las operaciones de asignación, inspección de matrices, inicialización de las constantes, que seran n+k. Además utiliza 2 veces la ordenación de python, que, aunque no se puede saber exactamente el número de operaciones que hará en el peor de los casos, revisando la documentación se puede descubrir que utiliza un algoritmo llamado **Timsort**, que es una variación muy optimizada de un merge sort que tiene un órden de complejidad $O(n \log{n})$ en el peor de los casos.<br><br>
En resumen, el órden de complejidad, al ser mayor que el del resto de operaciones, lo marcan las ordenaciones y queda en  $O(n \log{n})$<br><br>

En el caso del *greedy_algorithm*, sin entrar en tanto detalle, es fácil ver que el órden de complejidad será de $O(n^2)$

(*)Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

**Respuesta**

In [0]:
# Se define una función generadora de jornadas aleatorias a partir de la lista de equipos de la liga
# TEAMS es constante y forma parte del enunciado del problema
def generate_matchday():
  shuffled = TEAMS
  # Se ordenan aleatoriamente los equipos
  random.shuffle(shuffled)
  # Se retornan parejas de equipos que conformaran los partidos de la jornada
  return {i: shuffled[i * 2:(i + 1) * 2] for i in range((len(shuffled) + 1) // 2 )}

# Se define una función encargada de crear la matriz de audiencias posibles
# combinando los partidos recibidos con los posibles horarios.
# TIMETABLE_MULT Contiene los multiplicadores asociados a cada horario
def create_audience_matrix(matches_list):
  length = len(matches_list)
  audience_matrix = [ [0]*length for i in [0]*length ]
  audience_list = {}
  for m_it in range(length):
    audience_list[m_it] = match_audience(matches_list[m_it][0],matches_list[m_it][1])
    for t_it in range(len(TIMETABLE_MULT)):
      audience_matrix[m_it][t_it] = audience_list[m_it]*TIMETABLE_MULT[t_it][0]
      
  return audience_matrix, audience_list

(*)Aplica el algoritmo al juego de datos generado

**Respuesta**

In [0]:
# Se importa la función de cálculo de tiempos aportada en el foro
from time import time

def calcular_tiempo(f):
   
    def wrapper(*args, **kwargs):        
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper
  
# Se generan wrappers para los tres algoritmos
@calcular_tiempo
def wrapped_brute_force(audience_matrix):
  return brute_force(audience_matrix)
  
@calcular_tiempo
def wrapped_sort_algorithm(audience_list, audience_matrix):
  return sort_algorithm(audience_list, audience_matrix)

@calcular_tiempo
def wrapped_greedy_algorithm(audience_list, audience_matrix):
  return greedy_algorithm(audience_list, audience_matrix)

In [6]:
# Se genera una jornada aleatoriamente y se muestra
match_day = generate_matchday()
print("Partidos generados:")
print(match_day)

# Se genera la matriz de audiencias
audience_matrix, audience_list = create_audience_matrix(match_day)

# De cara a una representación más visual de los datos generados se montan unos dataframes
# y se muestra su combinación
audience_df = pd.DataFrame(audience_matrix, columns=np.array(TIMETABLE_MULT)[:,2])
match_df = pd.DataFrame(match_day).T
match_df.columns = ['T1', 'T2']
match_aud_df = pd.DataFrame(list(audience_list.values()))
match_aud_df.columns = ['BaseAud']
final_dataframe = match_df.join(match_aud_df).join(audience_df)
print("\nMatriz de posibles audiencias:")
print(final_dataframe)

# Las soluciones son una lista dónde la posición es el partido y el valor en esa posición
# el horario asociado al partido.

# Se soluciona el problema por fuerza bruta y se muestra el resultado
print("\nFUERZA BRUTA:")
best_solution, total_audience = wrapped_brute_force(audience_matrix)
print(best_solution)
print(total_audience[0])

# Se soluciona el problema con el algoritmo de ordenación de listas y se muestra el resultado
print("\nSORT ALGORITHM:")
best_solution, total_audience = wrapped_sort_algorithm(audience_list, audience_matrix)
print(best_solution)
print(total_audience)

# Se soluciona el problema con el algoritmo voraz y se muestra el resultado
print("\nGREEDY ALGORITHM:")
best_solution, total_audience = wrapped_greedy_algorithm(audience_list, audience_matrix)
print(best_solution)
print(total_audience)

Partidos generados:
{0: ['C', 'B'], 1: ['B', 'A'], 2: ['B', 'A'], 3: ['B', 'C'], 4: ['C', 'C'], 5: ['B', 'B'], 6: ['C', 'B'], 7: ['A', 'C'], 8: ['A', 'B'], 9: ['B', 'B']}

Matriz de posibles audiencias:
  T1 T2  BaseAud  V20 (0.4)  S12 (0.55)  S16 (0.7)  S18 (0.8)  S20 (1)  \
0  C  B     0.75      0.300      0.4125      0.525      0.600     0.75   
1  B  A     1.30      0.520      0.7150      0.910      1.040     1.30   
2  B  A     1.30      0.520      0.7150      0.910      1.040     1.30   
3  B  C     0.75      0.300      0.4125      0.525      0.600     0.75   
4  C  C     0.47      0.188      0.2585      0.329      0.376     0.47   
5  B  B     0.90      0.360      0.4950      0.630      0.720     0.90   
6  C  B     0.75      0.300      0.4125      0.525      0.600     0.75   
7  A  C     1.00      0.400      0.5500      0.700      0.800     1.00   
8  A  B     1.30      0.520      0.7150      0.910      1.040     1.30   
9  B  B     0.90      0.360      0.4950      0.630      0

- A veces los resultados no son exactamente iguales debido a que hay horarios y partidos que comparten valores de audiencia. El sumatorio de la audiencia máxima conseguible siempre será la misma, eso si.
- Curiosamente la mayoría de las veces el GREEDY ALGORITHM consigue las soluciones en mejores tiempos que el SORT ALGORITHM, seguramente debido a las constantes que se desconocen internas del sorted de python. Siendo un problema cerrado a 20 equipos, se podría afinar un poco más el GREEDY y aplicarlo.

(*)Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

**Respuesta**

https://ocw.ehu.eus/pluginfile.php/5838/mod_resource/content/1/6._transporte_asignacion.pdf<br>
https://en.wikipedia.org/wiki/Hungarian_algorithm<br>
https://www.uv.es/martinek/material/Tema6.pdf<br>
https://hackernoon.com/timsort-the-fastest-sorting-algorithm-youve-never-heard-of-36b28417f399<br>
Manual de la asignatura

(*)Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

**Respuesta**

De cara a hacer el problema más interesante, se podrían introducir modificaciones para asemejarlo un poco más a un caso más realista.<br><br>
La primera posibilidad sería la de añadir condicionantes al multiplicador horario según el equipo. Por razones demográficas es perfectamente plausible pensar que las aficiones de los distintos equipos se comportarán diferente en relación con los cambios horarios. Un equipo tendrá mejores audiencias en domingo que otro, o se podría hasta beneficiar de jugar por la tarde.<br><br>
Si se planteara de esta manera, se convertiría en un problema de asignación al uso al que se podría aplicar programación dinámica o el método húngaro referenciado arriba.<br><br>
Una siguiente evolución podría ser la de permitir repetición de horario. Aquí el problema se complicaría mucho más y el algoritmo óptimo seguramente dependería también de como se realizaran las condiciones.<br><br>
A primera vista, si se pusiera un máximo de dos partidos por horario (después que ya fuera inviable por razones de repartición de audiencia) y partiendo de que se conocería una solución próxima a la mejor (la encontrada multiplicando mejor horario por mejor partido), se podría intentar un algoritmo de ramificación y poda, esperando encontrar una buena estrategia de poda a partir de las condiciones comentadas.<br><br>
Si se permitiera y fuera plausible cualquier combinación, la complejidad escalaría de forma que lo más sensato parecería utilizar métodos heurísticos.